This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
# get api key from your .env file
import os
# Install a pip package (namely python-dotenv) in the current Jupyter kernel
import sys
!{sys.executable} -m pip install python-dotenv
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

##print(API_KEY)

Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import pprint as pp

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?start_date=2023-01-13&end_date=2023-01-13"
r = requests.get(url)
print("type(r):", type(r))

type(r): <class 'requests.models.Response'>


In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r_json = r.json()
pp.pprint(r_json)

{'dataset': {'collapse': None,
             'column_index': None,
             'column_names': ['Date',
                              'Open',
                              'High',
                              'Low',
                              'Close',
                              'Change',
                              'Traded Volume',
                              'Turnover',
                              'Last Price of the Day',
                              'Daily Traded Units',
                              'Daily Turnover'],
             'data': [],
             'database_code': 'FSE',
             'database_id': 6129,
             'dataset_code': 'AFX_X',
             'description': 'Stock Prices for Carl Zeiss Meditec (2020-11-02) '
                            'from the Frankfurt Stock Exchange.<br><br>Trading '
                            'System: Xetra<br><br>ISIN: DE0005313704',
             'end_date': '2020-12-01',
             'frequency': 'daily',
             'id': 

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

Proceeding step-by-step
1. Collect data from the Frankfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017
2. Convert the returned JSON object into a Python dictionary.

In [5]:
# Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017
# and convert the requests output into a Python dictionary.
url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&order=asc"
r = requests.get(url)
zeiss_2017 = r.json()
print("type(zeiss_2017):", type(zeiss_2017))

type(zeiss_2017): <class 'dict'>


3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [6]:
# Using the JSON nested structure printed above, we see that the entire
# dataset is nested under a "dataset" member. Within that nested dict,
# "Open" is the second list item (at index 1) of the "data" member.
#
# Let's start by reducing one level of nesting and pull out the list
# of data lists, so we have one data list for each trading day's data.
# We also print the first 10 data lists to make sure we have what
# we think we have.
#
zeiss_2017_data = zeiss_2017['dataset']['data']
print("type(zeiss_2017_data):", type(zeiss_2017_data), "\n")
print("first 10 days' data lists:\n", zeiss_2017_data[0:10])
print()
#
# Now use a list comprehension to get a list of the
# opening price for each trading day.
#
opens = [data[1] for data in zeiss_2017_data]
print("type(opens):", type(opens))
#
# Finally, let's find and print the highest and lowest values
# Note that None values need to be handled, as they trip up min() and max()
#
max2017open = max(i for i in opens if i is not None)
min2017open = min(i for i in opens if i is not None)
print("\nhighest opening price in 2017: {:.2f}".format(max2017open))
print("\nlowest opening price in 2017: {:.2f}".format(min2017open))

type(zeiss_2017_data): <class 'list'> 

first 10 days' data lists:
 [['2017-01-02', 34.99, 35.94, 34.99, 35.8, None, 44700.0, 1590561.0, None, None, None], ['2017-01-03', 35.9, 35.93, 35.34, 35.48, None, 70618.0, 2515473.0, None, None, None], ['2017-01-04', 35.48, 35.51, 34.75, 35.19, None, 54408.0, 1906810.0, None, None, None], ['2017-01-05', 35.02, 35.2, 34.73, 35.06, None, 48412.0, 1692326.0, None, None, None], ['2017-01-06', 34.91, 35.21, 34.91, 35.04, None, 27507.0, 964046.0, None, None, None], ['2017-01-09', 35.29, 35.35, 34.43, 34.67, None, 62225.0, 2157182.0, None, None, None], ['2017-01-10', 34.8, 34.98, 34.46, 34.91, None, 43976.0, 1528055.0, None, None, None], ['2017-01-11', 34.95, 36.0, 34.84, 35.42, None, 123530.0, 4369079.0, None, None, None], ['2017-01-12', 35.38, 35.38, 34.31, 34.9, None, 163860.0, 5703427.0, None, None, None], ['2017-01-13', 34.98, 34.98, 34.6, 34.85, None, 59367.0, 2065534.0, None, None, None]]

type(opens): <class 'list'>

highest opening price in 20

4. What was the largest change in any one day (based on High and Low price)?

In [7]:
# Similar to above, use a list comprehension to get a list of one-day change
# from the daily highs and lows.
changes = [data[2] - data[3] for data in zeiss_2017_data]
max2017change = max(changes)
print("\nmax 1-day price change in 2017: {:.2f}".format(max2017change))
# check
print("\ncheck:")
print("(looping through data lists, print any list with (day high) - (day low) > 2.5)")
for i in zeiss_2017_data :
  if i[2] - i[3] > 2.5 :
    print(i)


max 1-day price change in 2017: 2.81

check:
(looping through data lists, print any list with (day high) - (day low) > 2.5)
['2017-05-11', 43.4, 46.06, 43.25, 45.0, None, 189125.0, 8496322.0, None, None, None]


5. What was the largest change between any two days (based on Closing Price)?

In [8]:
# The API has filtering capabilities that can calculate a difference between
# data items on day t and t-1. Let's look at the closing prices (at data
# index 4) and apply the diff transformation.
#
url = "https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X.json?column_index=4&transform=diff&start_date=2017-01-01&end_date=2017-12-31&order=asc"
r = requests.get(url)
zeiss_2017_close_diff = r.json()
#
# As before, let's reduce the resulting nested dictionary structure by
# one level and pull out the list of daily data lists.
# We also print the first 10 data lists to make sure we have what
# we think we have.
#
zeiss_2017_change = zeiss_2017_close_diff['dataset']['data']
print("type(zeiss_2017_change):", type(zeiss_2017_change), "\n")
print("first 10 days' data lists:\n", zeiss_2017_change[0:10])
print()
#
# Now let's find the maximum *absolute* change.
# First, do a comprehension to replace changes by their absolute value.
# We'll do a dict comprehension in order to preserve the dates.
#
changes = {data[0] : abs(data[1]) for data in zeiss_2017_change}
max_change = round(max(changes.values()), 2)
print("\nmax day-over-day change in 2017: {:.2f}".format(max_change))
#
# Print out the date on which this maximum change occured.
# (Or dates, plural, if there were ties.)
# This could be done more efficiently, but the explicit for
# loop has the benefit of readability.
#
print("on")
for key, value in changes.items() :
  if round(value, 2) == max_change :
        print(key)

type(zeiss_2017_change): <class 'list'> 

first 10 days' data lists:
 [['2017-01-03', -0.32], ['2017-01-04', -0.29], ['2017-01-05', -0.13], ['2017-01-06', -0.020000000000003], ['2017-01-09', -0.37], ['2017-01-10', 0.24], ['2017-01-11', 0.51], ['2017-01-12', -0.52], ['2017-01-13', -0.049999999999997], ['2017-01-16', 0.22]]


max day-over-day change in 2017: 2.56
on
2017-08-09


6. What was the average daily trading volume during this year?

In [10]:
# The trading volume is at index 6 in the daily data lists.
# Let's collect them with a list comprehension and take
# the mean of the list.
#
##test_data = [ [1, 1, 6], [1, 1, 8], [1, 1, 9], [1, 1, None] ]
##test_vols = [data[2] for data in test_data]
##ave_vol = sum(test_vols) / len(test_vols)
##print("\nAverage volume in test: {:.1f}".format(ave_vol))
volumes = [data[6] for data in zeiss_2017_data]
ave_vol = sum(volumes) / len(volumes)
print("\nAverage volume in 2017: {:.1f}".format(ave_vol))


Average volume in 2017: 89124.3
